In [88]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
# import matplotlib.pyplot as plt
# import numpy as np
import os
from time import sleep

In [89]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [90]:
def login():
    """ Login to the Rapsodo Cloud
    """
    url = 'https://cloud.rapsodo.com/'
    driver = webdriver.Firefox()
    driver.get(url)

    username = driver.find_element_by_id("login_username")
    password = driver.find_element_by_id("login_password")

    username.send_keys(os.environ["RAPSODO_LOGIN"])
    password.send_keys(os.environ["RAPSODO_PASSWD"])

    driver.find_element_by_id("_login").click()    
    sleep(2)
    
    return driver

In [94]:
def pitcherStats():
    """ scrape pitcher's stats
            * current: naviagates to Super's stats
    """
    driver = login()
    
    table = driver.find_element_by_id('playerTable')
    sleep(1)
    playerList = table.find_element_by_tag_name('tbody')
    sleep(1)
    players = playerList.find_elements_by_tag_name('tr')
    sleep(1)
    
    elem = players[1].find_elements_by_tag_name('td')
    link = elem[-1].find_element_by_tag_name('a').get_attribute('href')
    
    driver.get(link)
    sleep(3)
    html = driver.page_source
    cols = scrapeStats(html)
    
    return cols

In [96]:
def scrapeStats(page):
    """ scrapes stats from page
            * current: returns table columns
    """
    soup = BeautifulSoup(page, 'html.parser')
    
    table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="shotTable")
    cols = [col.get_text() for col in table.findAll(lambda tag: tag.name=='th')]
    cols[-2] = 'Pitch Type'
    cols = cols[0:-1]

    return cols

In [97]:
pitcherStats()

[u'No',
 u'Date',
 u'Speed(MPH)',
 u'Spin(RPM)',
 u'True Spin(RPM)',
 u'Spin Eff.(%)',
 u'Spin Axis(hh:mm)',
 u'SpinConf.',
 u'VB',
 u'HB',
 u'StrikeX(in)',
 u'StrikeY(in)',
 u'Mode',
 u'Strike',
 'Pitch Type']